In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.shape

# 1. EDA

In [ ]:
train_df['question_text'].isnull().sum()

In [ ]:
train_df['question_text'].isna().sum()

Tập huấn luyện không có giá trị NULL.

In [ ]:
# Đổi tên cột 'question_text' -> 'question' 
train_df = train_df.rename({'question_text': 'question'}, axis=1)

In [ ]:
train_df.columns

Tập huấn luyện gồm 3 cột: ***qid***, ***question*** và ***target***.

In [ ]:
# Đếm số lượng hàng dữ liệu của mỗi nhãn.
train_df.target.value_counts()

In [ ]:
import matplotlib.ticker as ticker

ncount = train_df.shape[0]

plt.figure(figsize=(7, 5))

ax = sns.countplot(data=train_df, x='target')
plt.title('Distribution of Questions')
plt.xlabel('Number of Axles')

# Make twin axis
ax2=ax.twinx()

# Switch so count axis is on right, frequency on left
ax2.yaxis.tick_left()
ax.yaxis.tick_right()

# Also switch the labels over
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax2.set_ylabel('Frequency [%]')

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text

# Use a LinearLocator to ensure the correct number of ticks
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Fix the frequency range to 0-100
ax2.set_ylim(0,100)
ax.set_ylim(0,ncount)

# And use a MultipleLocator to ensure a tick spacing of 10
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# Need to turn the grid on ax2 off, otherwise the gridlines end up on top of the bars
ax2.grid(None)

Từ các số liệu trên, tập dữ liệu bao gồm 1225312 câu hỏi chất lượng, chiếm 93,8%. Trong khi đó, có tổng cộng 80810 câu hỏi kém chất lượng, chiếm 6.2% tập dữ liệu. Như vậy, tập dữ liệu của chúng ta rất không cân bằng. 

In [ ]:
pd.set_option('display.max_columns', None)
insincere_qes = train_df[train_df['target'] == 1]
print(insincere_qes[:5].question)

In [ ]:
pd.set_option('display.max_columns', None)
sincere_qes = train_df[train_df['target'] == 0]
print(sincere_qes[-5:].question)

## Words cloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
print('Word cloud của loại câu hỏi chất lượng.')
insincere_wordcloud = WordCloud(width=800, height=400, background_color ='black', min_font_size = 10).generate(str(train_df[train_df["target"] == 1]["question"]))
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.show();

In [ ]:
print('Word cloud của loại câu hỏi kém chất lượng.')
insincere_wordcloud = WordCloud(width=800, height=400, background_color ='black', min_font_size = 10).generate(str(train_df[train_df["target"] == 0]["question"]))
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.show();

## Feature engineering

In [ ]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

stop_words = stopwords.words('english')
stop_words.remove('not')
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [ ]:
contraction_dict = {"dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not", "cant": "cannot", "mustnt": "must not", "hasnt": "has not", "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not","can't": "cannot", "‘cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

# if contraction_dict.has_key('aya'):
#     print('yaya')
punctuation = string.punctuation 

In [ ]:
def replace_contractions(question):
    return [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in question]

In [ ]:
#Feature Engineering on train_df Data
def create_features(df_):
    """Retrieve from the text column the nb of : words, unique words, characters, stopwords,
    punctuations, upper/lower case char, title..."""
    
    df_["question_length"] = df_["question"].apply(lambda x: len(x))
    df_["nb_words"] = df_["question"].apply(lambda x: len(x.split()))
    df_["nb_unique_words"] = df_["question"].apply(lambda x: len(set(str(x).split())))
    df_["nb_chars"] = df_["question"].apply(lambda x: len(str(x)))
    df_["nb_stopwords"] = df_["question"].apply(lambda x : len([nw for nw in str(x).split() if nw.lower() in stop_words]))
    df_["nb_punctuation"] = df_["question"].apply(lambda x : len([np for np in str(x) if np in punctuation]))
    df_["nb_uppercase"] = df_["question"].apply(lambda x : len([nu for nu in str(x).split() if nu.isupper()]))
    df_["nb_lowercase"] = df_["question"].apply(lambda x : len([nl for nl in str(x).split() if nl.islower()]))
    df_["nb_title"] = df_["question"].apply(lambda x : len([nl for nl in str(x).split() if nl.istitle()]))
    return df_

In [ ]:
train_df = create_features(train_df)

In [ ]:
train_df.head()

**Số liệu thống kê của loại câu hỏi chất lượng.**

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train_df[train_df['target'] == 0].describe()

**Số liệu thống kê của loại câu hỏi kém chất lượng.**

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train_df[train_df['target'] == 1].describe()

In [ ]:
num_feat = ['question_length', 'nb_unique_words', 'nb_chars', 'nb_stopwords', \
            'nb_punctuation', 'nb_uppercase', 'nb_lowercase', 'nb_title', 'target'] 

df_sample = train_df[num_feat].sample(n=round(train_df.shape[0]/6), random_state=42)

plt.figure(figsize=(16,10))
sns.pairplot(data=df_sample, hue='target')
plt.show()

Qua hai bảng số liệu, ta có thể thấy độ dài trung bình của loại câu hỏi kém chất lượng ngắn hơn loại câu hỏi chất lượng. Các features khác cũng tương tự như vậy, có thể là do chúng chịu ảnh hưởng của sự chênh lệch độ dài trên. 

In [ ]:
mask = np.zeros_like(train_df[num_feat].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(10, 10))
plt.title('Question features Correlation Matrix',fontsize=20)

sns.heatmap(train_df[num_feat].corr(),square=True, linewidths=0.25,vmax=0.7,cmap="YlGnBu",
            linecolor='w',annot=True,annot_kws={"size":10},mask=mask,cbar_kws={"shrink": .9});

In [ ]:
plt.figure(figsize=(15, 5))

target_correlation = train_df.corr()['target'][1:]
plt.plot(target_correlation)

Có thể thấy, độ dài của một câu hỏi phân biệt loại của nó tốt nhất, tuy nhiên, tỉ lệ này là khá ít (chỉ 0.18). Tóm lại, mọi features đều có sự liên quan nhất định tới phân loại của câu hỏi, mặc dù sự liên quan này là không cao.

# Data preprocessing

In [ ]:
def qes_preprocessing(qes):
    # Data cleaning:
    qes = re.sub(re.compile('<.*?>'), '', qes)
    qes = re.sub('[^A-Za-z0-9]+', ' ', qes)

    # Lowercase:
    qes = qes.lower()

    # Tokenization:
    tokens = word_tokenize(qes)

    # Contractions replacement:
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]

    # Stop words removal:
    tokens = [w for w in tokens if w not in stop_words]

    # Stemming:
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Lemmatization:
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    # Join words after preprocessed:
    qes = ' '.join(tokens) 

    return qes

In [ ]:
tqdm.pandas()
train_df['preprocessed_questions'] = train_df['question'].progress_apply(qes_preprocessing)

# Train model on training set

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,4), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=300, verbose=1, n_jobs=-1))])

X = train_df['preprocessed_questions']
y = train_df['target']

In [ ]:
lr_model = pipeline.fit(X, y)

# Load test data and make prediction

In [ ]:
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_df['preprocessed'] = test_df['question_text'].apply(qes_preprocessing)

In [ ]:
predictions = lr_model.predict(test_df['preprocessed'])

In [ ]:
test_df['prediction'] = predictions
results = test_df[['qid', 'prediction']]
results.to_csv('submission.csv', index=False)

In [ ]:
results.head()